# Linear Regression Project

Project from the Udemy course "Spark and Python for Big Data with PySpark"

Aim: to give accurate estimates of how many crew members a ship will require.

Data available:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.

    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('cruise').getOrCreate()

In [4]:
data = spark.read.csv("/FileStore/tables/cruise_ship_info.csv",inferSchema=True,header=True)

In [5]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 20 rows

In [6]:
data.columns

Out[7]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [7]:
data.describe().show()

+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
summary|Ship_name|Cruise_line| Age| Tonnage| passengers| length| cabins|passenger_density| crew|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
 count| 158| 158| 158| 158| 158| 158| 158| 158| 158|
 mean| Infinity| null|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
 stddev| NaN| null| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
 min|Adventure| Azamara| 4| 2.329| 0.66| 2.79| 0.33| 17.7| 0.59|
 max|Zuiderdam| Windstar| 48| 220.0| 54.0| 11.82| 27.0| 71.43| 21.0|
+-------+---------+-----------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+

In [8]:
data.groupBy('Cruise_line').count().show()

+-----------------+-----+
 Cruise_line|count|
+-----------------+-----+
 Costa| 11|
 P&O| 6|
 Cunard| 3|
Regent_Seven_Seas| 5|
 MSC| 8|
 Carnival| 22|
 Crystal| 2|
 Orient| 1|
 Princess| 17|
 Silversea| 4|
 Seabourn| 3|
 Holland_American| 14|
 Windstar| 3|
 Disney| 2|
 Norwegian| 13|
 Oceania| 3|
 Azamara| 2|
 Celebrity| 10|
 Star| 6|
 Royal_Caribbean| 23|
+-----------------+-----+

In [9]:
from pyspark.ml.feature import StringIndexer

In [10]:
#Encoding categorical feature Cruise_line 
indexer = StringIndexer(inputCol="Cruise_line", outputCol="Cruise_lineIndex")
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------+
 Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_lineIndex|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55| 16.0|
Celebration| Carnival| 26| 47.262| 14.86| 7.22| 7.43| 31.8| 6.7| 1.0|
 Conquest| Carnival| 11| 110.0| 29.74| 9.53| 14.88| 36.99|19.1| 1.0|
 Destiny| Carnival| 17| 101.353| 26.42| 8.92| 13.21| 38.36|10.0| 1.0|
 Ecstasy| Carnival| 22| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Elation| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Fantasy| Carnival| 23| 70.367| 20.56| 8.55| 10.22| 34.23| 9.2| 1.0|
Fascination| Carnival| 19| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Freedom| Carnival| 6|110.23899999999999| 37.0| 9.51| 14.87| 29.79|11.5| 1.0|
 Glory| Carnival| 10| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Holiday| Carnival| 28| 46.052| 14.52| 7.27| 7.26| 31.72| 6.6| 1.0|
Imagination| Carnival| 18| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
Inspiration| Carnival| 17| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Legend| Carnival| 11| 86.0| 21.24| 9.63| 10.62| 40.49| 9.3| 1.0|
 Liberty*| Carnival| 8| 110.0| 29.74| 9.51| 14.87| 36.99|11.6| 1.0|
 Miracle| Carnival| 9| 88.5| 21.24| 9.63| 10.62| 41.67|10.3| 1.0|
 Paradise| Carnival| 15| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
 Pride| Carnival| 12| 88.5| 21.24| 9.63| 11.62| 41.67| 9.3| 1.0|
 Sensation| Carnival| 20| 70.367| 20.52| 8.55| 10.2| 34.29| 9.2| 1.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------------+
only showing top 20 rows

In [11]:
from pyspark.ml.regression import LinearRegression

In [12]:
#Setting up the dataframe for machine learning
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=['Cruise_lineIndex', 'Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density'],
    outputCol='features')

In [13]:
output = assembler.transform(indexed)

In [14]:
output.head(1)

Out[18]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_lineIndex=16.0, features=DenseVector([16.0, 6.0, 30.277, 6.94, 5.94, 3.55, 42.64]))]

In [15]:
final_data = output.select(["features",'crew'])
final_data.show()

+--------------------+----+
 features|crew|
+--------------------+----+
[16.0,6.0,30.2769...|3.55|
[16.0,6.0,30.2769...|3.55|
[1.0,26.0,47.262,...| 6.7|
[1.0,11.0,110.0,2...|19.1|
[1.0,17.0,101.353...|10.0|
[1.0,22.0,70.367,...| 9.2|
[1.0,15.0,70.367,...| 9.2|
[1.0,23.0,70.367,...| 9.2|
[1.0,19.0,70.367,...| 9.2|
[1.0,6.0,110.2389...|11.5|
[1.0,10.0,110.0,2...|11.6|
[1.0,28.0,46.052,...| 6.6|
[1.0,18.0,70.367,...| 9.2|
[1.0,17.0,70.367,...| 9.2|
[1.0,11.0,86.0,21...| 9.3|
[1.0,8.0,110.0,29...|11.6|
[1.0,9.0,88.5,21....|10.3|
[1.0,15.0,70.367,...| 9.2|
[1.0,12.0,88.5,21...| 9.3|
[1.0,20.0,70.367,...| 9.2|
+--------------------+----+
only showing top 20 rows

In [16]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 108|
 mean| 7.652500000000007|
 stddev|3.6861417980240794|
 min| 0.59|
 max| 21.0|
+-------+------------------+

In [18]:
test_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 50|
 mean| 8.1002|
 stddev|3.0850386098863964|
 min| 1.8|
 max| 13.6|
+-------+------------------+

In [19]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='crew')

In [20]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data)

In [21]:
test_results = lrModel.evaluate(test_data)

In [22]:
test_results.residuals.show()

+--------------------+
 residuals|
+--------------------+
 -1.7830053972068978|
 -0.6016186101071295|
 -0.5449459088503446|
 -1.343217781026997|
 -1.42325025186123|
 -1.6111724175860385|
 -0.5512048884202692|
 -1.0198152025612917|
 -0.877721396337499|
 -0.7532856310943323|
 0.6962168448298307|
 -0.7333505727627951|
 0.1419157216701894|
 -0.8321971533057866|
 0.4940663158132832|
 0.5439039616421244|
 0.5638390199736616|
 0.4846602088341063|
 -0.9332052826514978|
-0.23219148495270225|
+--------------------+
only showing top 20 rows

In [23]:
test_results.rootMeanSquaredError

Out[32]: 0.7661058985999392

In [24]:
test_results.r2

Out[33]: 0.9370739705350807

In [25]:
from pyspark.sql.functions import corr

In [26]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [27]:
data.select(corr('crew', 'cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+